### Data Ingestion


In [3]:
### document structure

from langchain_core.documents import Document

In [4]:
doc=Document(
    page_content="This is the content of the document.", 
    metadata={
        "source": "user_manual.pdf",
        "pages":1,
        "author":"John Doe"}
    )

doc

Document(metadata={'source': 'user_manual.pdf', 'pages': 1, 'author': 'John Doe'}, page_content='This is the content of the document.')

In [5]:
## create a simple txt file

import os
os.makedirs("../data/text_files", exist_ok=True)

In [6]:
sample_texts={
    "../data/text_files/doc1.txt": """RAG Intoduction
    Retrieval-Augmented Generation (RAG) is a technique used to enhance large language models by combining text generation with information retrieval. Instead of relying only on the knowledge stored inside the model during training, RAG systems dynamically fetch relevant information from external data sources before generating a response.

In a typical RAG pipeline, documents such as PDFs, text files, or web pages are first loaded and converted into a structured format. These documents are then split into smaller chunks to ensure efficient processing. Each chunk is transformed into a numerical representation called an embedding, which captures the semantic meaning of the text.

When a user asks a question, the query is also converted into an embedding and compared against the stored document embeddings using a vector similarity search. The most relevant chunks are retrieved and passed to a language model as additional context. This allows the model to produce more accurate, up-to-date, and context-aware answers.

RAG is especially useful in applications like chatbots, question-answering systems, and enterprise knowledge assistants, where factual accuracy and domain-specific information are critical. By grounding model responses in retrieved documents, RAG helps reduce hallucinations and improves trustworthiness."""

}

for file_path, content in sample_texts.items():
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(content)

In [9]:
##TextLoader
from langchain_community.document_loaders import TextLoader

loader= TextLoader("../data/text_files/doc1.txt", encoding="utf-8")
documents=loader.load()
documents


[Document(metadata={'source': '../data/text_files/doc1.txt'}, page_content='RAG Intoduction\n    Retrieval-Augmented Generation (RAG) is a technique used to enhance large language models by combining text generation with information retrieval. Instead of relying only on the knowledge stored inside the model during training, RAG systems dynamically fetch relevant information from external data sources before generating a response.\n\nIn a typical RAG pipeline, documents such as PDFs, text files, or web pages are first loaded and converted into a structured format. These documents are then split into smaller chunks to ensure efficient processing. Each chunk is transformed into a numerical representation called an embedding, which captures the semantic meaning of the text.\n\nWhen a user asks a question, the query is also converted into an embedding and compared against the stored document embeddings using a vector similarity search. The most relevant chunks are retrieved and passed to a 

In [11]:
## directory loader
from langchain_community.document_loaders import DirectoryLoader

dir_loader= DirectoryLoader("../data/text_files", 
glob="*.txt",  #Pattern to match files
loader_cls=TextLoader, #Loader class to use
loader_kwargs={"encoding":"utf-8"},#Loader specific arguments
show_progress=False
)

documents=dir_loader.load()
documents

[Document(metadata={'source': '..\\data\\text_files\\doc1.txt'}, page_content='RAG Intoduction\n    Retrieval-Augmented Generation (RAG) is a technique used to enhance large language models by combining text generation with information retrieval. Instead of relying only on the knowledge stored inside the model during training, RAG systems dynamically fetch relevant information from external data sources before generating a response.\n\nIn a typical RAG pipeline, documents such as PDFs, text files, or web pages are first loaded and converted into a structured format. These documents are then split into smaller chunks to ensure efficient processing. Each chunk is transformed into a numerical representation called an embedding, which captures the semantic meaning of the text.\n\nWhen a user asks a question, the query is also converted into an embedding and compared against the stored document embeddings using a vector similarity search. The most relevant chunks are retrieved and passed to

In [14]:
from reportlab.lib.pagesizes import LETTER
from reportlab.pdfgen import canvas
import os

# Ensure folder exists
os.makedirs("../data/pdf_files", exist_ok=True)

# ---------- PDF 1 ----------
pdf1_path = "../data/pdf_files/rag_intro.pdf"
c1 = canvas.Canvas(pdf1_path, pagesize=LETTER)

c1.drawString(72, 750, "Introduction to Retrieval-Augmented Generation (RAG)")
c1.drawString(72, 720, "RAG combines information retrieval with text generation.")
c1.drawString(72, 690, "It improves factual accuracy by grounding LLMs in documents.")

c1.save()

# ---------- PDF 2 ----------
pdf2_path = "../data/pdf_files/rag_pipeline.pdf"
c2 = canvas.Canvas(pdf2_path, pagesize=LETTER)

c2.drawString(72, 750, "RAG Pipeline Overview")
c2.drawString(72, 720, "1. Load documents (PDFs, text files, etc.)")
c2.drawString(72, 690, "2. Chunk documents")
c2.drawString(72, 660, "3. Generate embeddings")
c2.drawString(72, 630, "4. Retrieve relevant chunks for a query")

c2.save()

print("PDF files created successfully.")


PDF files created successfully.


In [15]:
from langchain_community.document_loaders import PyPDFLoader,PyMuPDFLoader

dir_loader= DirectoryLoader(
    "../data/pdf_files", 
    glob="*.pdf",  #Pattern to match files
    loader_cls=PyMuPDFLoader, #Loader class to use
    #loader_cls=PyPDFLoader,
   
    show_progress=False
)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'ReportLab PDF Library - (opensource)', 'creator': 'anonymous', 'creationdate': '2026-01-24T10:59:30+05:00', 'source': '..\\data\\pdf_files\\rag_intro.pdf', 'file_path': '..\\data\\pdf_files\\rag_intro.pdf', 'total_pages': 1, 'format': 'PDF 1.3', 'title': 'untitled', 'author': 'anonymous', 'subject': 'unspecified', 'keywords': '', 'moddate': '2026-01-24T10:59:30+05:00', 'trapped': '', 'modDate': "D:20260124105930+05'00'", 'creationDate': "D:20260124105930+05'00'", 'page': 0}, page_content='Introduction to Retrieval-Augmented Generation (RAG)\nRAG combines information retrieval with text generation.\nIt improves factual accuracy by grounding LLMs in documents.'),
 Document(metadata={'producer': 'ReportLab PDF Library - (opensource)', 'creator': 'anonymous', 'creationdate': '2026-01-24T10:59:30+05:00', 'source': '..\\data\\pdf_files\\rag_pipeline.pdf', 'file_path': '..\\data\\pdf_files\\rag_pipeline.pdf', 'total_pages': 1, 'format': 'PDF 1.3', 'title': 'un

In [18]:
print(type(pdf_documents))
print(type(pdf_documents[0]))

<class 'list'>
<class 'langchain_core.documents.base.Document'>


### embedding and vectorDB

In [3]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any,Tuple
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\OM\Desktop\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pip install sentence-transformers


Note: you may need to restart the kernel to use updated packages.


c:\Users\OM\Desktop\RAG\.venv\Scripts\python.exe: No module named pip


In [5]:
class EmbeddingManager:
    """ Handeles embedding generation using SentenceTransformer models. """
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager with a specified SentenceTransformer model.
        Args:
            model_name (str): HuggingFace Model name for sentence embeddings.

        """
        self.model_name = model_name
        self.model = None
        self._load_model()


    def _load_model(self):
        """ Load the SentenceTransformer model. """
        try:
            print(f"Loading Embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully.Embedding Dimension:{self.model.get_sentence_embedding_dimension()} ")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise e
        

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts.
        Args:
            texts (List[str]): List of input texts to embed.

        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dimension)
        """
        if not self.model:
            raise ValueError("Embedding model is not loaded.")
        
        print(f"Generating embeddings for {len(texts)} texts....")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings
    


    ### initializing the embedding manager
embedding_manager= EmbeddingManager()
embedding_manager

    

Loading Embedding model: all-MiniLM-L6-v2
Model loaded successfully.Embedding Dimension:384 


### VectorStore

In [ ]:
class VectorStore:
    """ Manages storage and retrieval of document embeddings using ChromaDB. """
    def __init__(self, collection_name: str = "pdf_documents",persist_directory: str="../data/vector_store"):
        """
        Initialize the vector store with a specified collection name.
        Args:
            collection_name (str): Name of the ChromaDB collection.
            persist_directory (str): Directory to persist vector store.
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()


    def _initialize_chromadb(self):
        """ Initialize ChromaDB client and collection. """
        try:
            print("Initializing ChromaDB client...")
            self.client = chromadb.Client(Settings(
                chroma_db_impl="duckdb+parquet",
                persist_directory="./chroma_db_data"
            ))
            self.collection = self.client.get_or_create_collection(name=self.collection_name)
            print(f"ChromaDB collection '{self.collection_name}' is ready.")
        except Exception as e:
            print(f"Error initializing ChromaDB: {e}")
            raise e